In [4]:
import sys
import json
import requests
from urllib.request import urlopen
import csv
import os
import os.path

def get_prot_name(filename):
    
    # Print the description lines of a BLASTX file
    blast_file = open(filename, 'r')
    line = blast_file.readline()

    # Sanity check that this is a BLASTX output text file
    if not line.startswith("BLASTX"):
        #raise TypeError("Not a BLASTX file: %r" % version)
        print("Not a BLASTX file: %r" % version)
        print(None)

    content = blast_file.readlines()
    lines = []

    for line in content:

        if line.startswith(">"):
            prot_name = line[1:].strip('\n ')
            prot_name = prot_name[:prot_name.find(' [')]
            prot_name = prot_name[prot_name.find(' '):].replace(' ', '+')[1:]
            lines.append(prot_name)

        if not line:
            # End of file - this should not happen
            print("Could not find the description section")
            print(None)

    unknow_list = ['hypothetical', 'predicted', 'uncharacterized', 'unknown', 'unnamed']
    prot_names = []

    # Return known proteins
    for line in lines:
        if not any(term in line.lower() for term in unknow_list):
            prot_names.append(line)
            
    prot_names = list(dict.fromkeys(prot_names))
    return prot_names
        

def get_full_anotation(prot_name):
    
    print(prot_name.replace('+', ' '))

    # First get the Uniprot data
    html = urlopen('https://www.uniprot.org/uniprot/?query=%22' + str(prot_name) + '%22,&columns=id,entry%20name,go&format=tab&sort=score').readlines()

    go_ids = set()
    go_terms = []
    gene_id = ''

    if html == []:
        print('No GO terms found for protein')
        return gene_id, go_ids
        
    else:
        if str(html).find('[GO:') != -1:
            decoded_html = html[1].decode("utf-8")
            go_terms = decoded_html.strip('\n').split('\t', 2)[-1].split(';')
            gene_id = decoded_html.strip('\n').split()[0]

        #fazer para cada go_item e so guardar o go term
        for term in go_terms:
            line = term.split()
            go_term = line[-1].replace('[', '').replace(']', '')
            go_ids.add(go_term)

    return gene_id, go_ids


def get_ancestors(go_ids):

    # Construct the URL for the QuickGO Request with all those GO identifiers
    the_url = 'https://www.ebi.ac.uk/QuickGO/services/ontology/go/terms/'
    the_url += ','.join(go_ids)
    the_url += '/ancestors?relations=is_a'

    # Make the request and parse the results
    response = requests.get(the_url)
    anc_data = json.loads(response.text)

    try:
        anc_data['results']
        results = anc_data['results']

        ancestors = set()
        for record in results:
            ancestors.update(record['ancestors'])

    except:
        ancestors = 'Something went wrong.'
        
    return ancestors


def get_all_go_terms(f_path):

    file_list = []
    prot_list = []
    for file in os.listdir(f_path):
        file_list.append(f_path + file)
    
    if os.path.exists('annotations.txt'):
        with open ('annotations.txt', 'r') as f_txt:
            all_go_terms = f_txt.read().split('\n')
        f_txt.close()
    else:
        all_go_terms = []
        
    for filename in file_list:
        
        trinity_gene = filename.replace(f_path, "").replace('_blastx', "")
        print(trinity_gene)
        
        if os.path.exists('go_output.csv'):
            
            if trinity_gene not in get_gene_list('go_output.csv'):
                prot_list = get_prot_name(filename)
            else:
                print(trinity_gene, ': Gene already annotated.')
                continue

        else:
            prot_list = get_prot_name(filename)
            
        if prot_list == None:
            continue
        
        if prot_list != []:
            
            for prot in prot_list:
                gene_id, anotations = get_full_anotation(prot)
                
                if gene_id == '':
                    continue
                
                if anotations == 'Something went wrong.' or anotations == 'No GO terms found for protein':
                    continue
                    
                else:
                    with open('go_output.csv', mode='a+') as f_csv:
                        
                        if trinity_gene not in get_gene_list('go_output.csv'):
                            csv_row = [trinity_gene, gene_id, prot.replace('+', ' '), str(anotations)]
                            writer = csv.writer(f_csv, delimiter=',')
                            writer.writerow(csv_row)
                        
                            with open ('annotations.txt', 'a+') as f_txt:
                                
                                for go_term in anotations:
                                    f_txt.write('%s\n' % go_term)
                            f_txt.close()
                            
                            all_go_terms.extend(anotations)
                            print('GO-terms found:', len(all_go_terms))
                            
                    f_csv.close()
                    break
        
    return all_go_terms


def get_term_info(go_id):
    the_url = 'https://www.ebi.ac.uk/QuickGO/services/ontology/go/terms/' + go_id
    response = requests.get(the_url)
    data = json.loads(response.text)
    
    record = data['results'][0] # Get the first result, because we are only requesting one!
    
    # Return a dictionary that contains the information of this GO term
    go_info = {
        'go_id': record['id'],
        'aspect': record['aspect'],
        'name': record['name'],
        'definition': record['definition']['text'],
        'obsolete': record['isObsolete'],
    }
    
    return go_info


def count_terms(all_go_terms, term_counts, n_freq):
    
    freq_go = []
    geral_terms = ['biological_process', 'cellular_component', 'molecular_function']
    aspect_dic = {'BP':0, 'MF':0, 'CC':0}

    for go_id in all_go_terms:
        go_info = get_term_info(go_id)
        print('Frequent GO-terms (at least ' + str(n_freq) + ' ')
        if term_counts[go_id] >= n_freq:
            if go_info['name'] not in geral_terms:
                freq_go.append('Aspect:' + go_info['aspect'] + '// Name:' + go_info['name'])
                print('Aspect:' + go_info['aspect'] + '// Name:' + go_info['name'])
                
        if go_info['aspect'] == 'molecular_function':
            aspect_dic['MF'] += 1
        elif get_term_info(go_id)['aspect'] == 'cellular_component':
            aspect_dic['CC'] +=1
        else:
            aspect_dic['BP'] +=1
    
    freq_go = list(dict.fromkeys(freq_go))
    
    print('Most frequent go-terms (present in at least ' + str(n_freq) + ' proteins):')
    print (freq_go)

    return aspect_dic


def get_wanted_terms(all_go_terms,term):
    
    dic_go = {}
    for go_id in all_go_terms:
        dic_go[go_id] = term_counts[go_id]
    
    count = {}
    for key in dic_go:
        name = get_term_info(key)['name']
        if term in name:
            count[name] = dic_go[key]
    return count


def get_gene_list(filename):
    gene_list = []
    with open(filename, 'r') as f_csv:
        csv_reader = csv.reader(f_csv, delimiter=',')
        lines_list = list(csv_reader)[1:]
        for line in lines_list:
            gene_list.append(line[0])
    return(gene_list)

In [431]:
get_full_anotation('early+nodulin-like+protein+1')

early nodulin-like protein 1


('Q9T076',
 {'GO:0005773',
  'GO:0005829',
  'GO:0005886',
  'GO:0009055',
  'GO:0009507',
  'GO:0031225',
  'GO:0046658',
  'GO:0048046',
  'GO:0099503'})

In [432]:
annotations = get_all_go_terms('Blast/blastx_out/')

early nodulin-like protein 1
GO-terms found: 9
UDP-glycosyltransferase 90A1-like
GO-terms found: 10
acyl-CoA N-acyltransferases super family protein
GO-terms found: 11
COBRA-like protein 4
GO-terms found: 22
DUF1118 domain-containing protein, partial
No GO terms found for protein
Cleavage and polyadenylation specificity factor subunit lik
No GO terms found for protein
ras GTPase-activating protein-binding protein 1-like
GO-terms found: 89
glutathione S-transferase
GO-terms found: 100
RecName: Full=CASP-like protein 1F1; Short=VvCASPL1F
No GO terms found for protein
DUF588 domain-containing protein
GO-terms found: 105
putative wall-associated receptor kinase-like 16
GO-terms found: 109
3-oxo-Delta(4,5)-steroid 5-beta-reductase-like
No GO terms found for protein
progesterone 5-beta-reductase 1
GO-terms found: 116
probable disease resistance protein At5g66900
GO-terms found: 122
protein NIM1-INTERACTING 1-like
GO-terms found: 133
kinesin heavy chain
GO-terms found: 161
50S ribosomal prote

In [340]:
term_counts = {term: annotations.count(term) for term in annotations}
term_counts

{'GO:0003824': 2,
 'GO:0044003': 1,
 'GO:0005575': 3,
 'GO:0016746': 1,
 'GO:0044248': 1,
 'GO:0017001': 1,
 'GO:1901136': 1,
 'GO:0035821': 1,
 'GO:0043655': 1,
 'GO:0016407': 1,
 'GO:1901657': 1,
 'GO:0051701': 1,
 'GO:0065007': 3,
 'GO:0043226': 1,
 'GO:1903561': 1,
 'GO:0051707': 1,
 'GO:0075136': 1,
 'GO:0048518': 1,
 'GO:0071704': 1,
 'GO:0016747': 1,
 'GO:0052572': 1,
 'GO:0052173': 1,
 'GO:0052040': 1,
 'GO:0033643': 1,
 'GO:0031349': 1,
 'GO:0016137': 1,
 'GO:0009987': 3,
 'GO:0017144': 1,
 'GO:0008080': 1,
 'GO:0048583': 1,
 'GO:0044403': 1,
 'GO:0044121': 1,
 'GO:0042802': 1,
 'GO:0044419': 1,
 'GO:0033646': 1,
 'GO:0050789': 3,
 'GO:0050896': 2,
 'GO:0052200': 1,
 'GO:0110165': 3,
 'GO:0050794': 3,
 'GO:0052032': 1,
 'GO:1901575': 1,
 'GO:0052167': 1,
 'GO:0044237': 1,
 'GO:0043227': 1,
 'GO:0034069': 1,
 'GO:0003674': 3,
 'GO:0043067': 1,
 'GO:0008150': 3,
 'GO:0009056': 1,
 'GO:0046677': 1,
 'GO:0016999': 1,
 'GO:0052164': 1,
 'GO:0044110': 1,
 'GO:0051704': 1,
 'GO:00306

In [ ]:
get_ancestors(annotations)

In [356]:
aspect_dic = count_terms(annotations, term_counts, 30)

Frequent GO-terms (at least 3 
Frequent GO-terms (at least 3 
Frequent GO-terms (at least 3 
Frequent GO-terms (at least 3 
Frequent GO-terms (at least 3 
Frequent GO-terms (at least 3 
Frequent GO-terms (at least 3 
Frequent GO-terms (at least 3 
Frequent GO-terms (at least 3 


KeyboardInterrupt: 

In [ ]:
get_wanted_terms(annotations, 'membrane')

In [ ]:
get_wanted_terms(annotations, 'hydrolase')

In [ ]:
get_wanted_terms(annotations, 'stress')

In [ ]:
get_wanted_terms(annotations, 'photosynthesis')

In [ ]:
get_wanted_terms(annotations, 'growth')

In [ ]:
get_wanted_terms(annotations, 'redox')

In [433]:
filename = 'Blast/blastx_out/TRINITY_DN222_c1_g1_blastx'
trinity_gene = filename.replace('Blast/blastx_out/', '').replace('_blastx', '')
print(trinity_gene)

TRINITY_DN222_c1_g1


In [8]:
for protein in proteins:
    go_list = []
    for go_term in protein_go_terms:
        get_term_info(go_term)['name']
    write.row(gene_id, prot_id, prot_name, FC, cc, mf, bp)
    #or save to a dictionary and write everything in the end

'biological regulation'